In [2]:
import os
import cv2
import numpy as np
import splitfolders
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten
import tensorflow_wavelets.Layers.DWT as DWT
import tensorflow_wavelets.Layers.DTCWT as DTCWT
import tensorflow_wavelets.Layers.DMWT as DMWT

In [ ]:
with open("./dataset/train.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.split(" ")
        category = line[-2]
        filename = line[-3]
        
        try:
            if category == "positive":
                os.replace(f"./dataset/train/{filename}", f"./dataset/data/positive/{filename}")
            elif category == "negative":
                os.replace(f"./dataset/train/{filename}", f"./dataset/data/negative/{filename}")
        except FileNotFoundError:
            pass

In [ ]:
with open("./dataset/test.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.split(" ")
        category = line[-2]
        filename = line[-3]
        
        try:
            if category == "positive":
                os.replace(f"./dataset/test/{filename}", f"./dataset/data/positive/{filename}")
            elif category == "negative":
                os.replace(f"./dataset/test/{filename}", f"./dataset/data/negative/{filename}")
        except FileNotFoundError as e:
            pass

In [ ]:
if os.path.exists("./dataset/data"):
    os.renames("./dataset/data", "./data")

In [ ]:
if not os.path.exists(os.path.join(os.getcwd(), './data_split')):
    splitfolders.ratio("./data", output="./data_split", seed=1337, ratio=(.7, .2, .1), group_prefix=None)

In [ ]:
vgg19 = VGG19(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
vgg19.trainable = False

In [ ]:
# Flatten and output layer
flatten_layer = Flatten()
prediction_layer = Dense(2, activation='softmax')

# the final model
model = Sequential([
    DWT.DWT(concat=1),
    vgg19,
    flatten_layer,
    prediction_layer
])

model.build(input_shape=((None, 224, 224, 3)))

# view the structure of the model
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [ ]:
# for training data
train_datagen = ImageDataGenerator(
    # rescaling pixel values
    rescale=1./255,
    
    # augmenting the data
    shear_range = 0.2,
    zoom_range = 0.4,
    horizontal_flip = True
)

# for cross validation data
val_datagen = ImageDataGenerator(
    # rescaling pixel values
    rescale=1./255,
)

# obtaining training dataset
training_set = train_datagen.flow_from_directory(
    './data_split/train/',
     target_size=(224, 224),
     batch_size=32,
     class_mode='categorical'
)

# obtaining cross-validation dataset
validation_set = val_datagen.flow_from_directory(
    './data_split/val/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=10,
    batch_size=16,
)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("slp_model.tflite", "wb") as m:
    m.write(tflite_model)
    
model.save("slp_model.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

plt.plot(epochs, acc, label='Training accuracy')
plt.plot(epochs, val_acc, label='Cross validation accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Cross validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
from keras.models import load_model

model = load_model("./model.h5")

In [ ]:
model.evaluate(test_set)

In [ ]:
y_pred = model.predict(test_set)

In [ ]:
labels_pred = []

for i in range(len(y_pred)):
#     print(np.argmax(y_pred[i]))
    labels_pred.append(np.argmax(y_pred[i]))

In [ ]:
labels = test_set.labels.tolist()

In [ ]:
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(labels, labels_pred)

In [ ]:
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cf_matrix.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')